In [ ]:
# -*- encoding: utf-8 -*-
"""
==============
Classification
==============
The following example shows how to fit a simple classification model with
*auto-sklearn*.
after install autosklearn, please restart runtime for updating sklearn version. 
"""

import sklearn.datasets
import sklearn.metrics

import autosklearn.classification
import threading
import time
from smac.runhistory.runhistory import RunHistory
from IPython.display import clear_output
############################################################################
# Data Loading
# ============

X, y = sklearn.datasets.load_breast_cancer(return_X_y=True)
X_train, X_test, y_train, y_test = \
    sklearn.model_selection.train_test_split(X, y, random_state=1)

############################################################################
# Build and fit a regressor
# =========================

tmp_runhistory = RunHistory()

def get_smac_object_callback(budget_type):
    def get_smac_object(
        scenario_dict,
        seed,
        ta,
        ta_kwargs,
        metalearning_configurations,
        n_jobs,
        dask_client,
    ):
        from smac.facade.hyperband_facade import HB4AC
        from smac.facade.roar_facade import ROAR
        from smac.facade.smac_ac_facade import SMAC4AC
        from smac.intensification.successive_halving import SuccessiveHalving
        from smac.scenario.scenario import Scenario

        if n_jobs > 1 or (dask_client and len(dask_client.nthreads()) > 1):
            raise ValueError("Please make sure to guard the code invoking Auto-sklearn by "
                             "`if __name__ == '__main__'` and remove this exception.")

        scenario = Scenario(scenario_dict)
        if len(metalearning_configurations) > 0:
            default_config = scenario.cs.get_default_configuration()
            initial_configurations = [default_config] + metalearning_configurations
        else:
            initial_configurations = None

        ta_kwargs['budget_type'] = budget_type

        return HB4AC(
            scenario=scenario,
            rng=seed,
            tae_runner=ta,
            tae_runner_kwargs=ta_kwargs,
            initial_design=None,
            initial_configurations=initial_configurations,
            runhistory=tmp_runhistory,
            run_id=seed,
            intensifier=SuccessiveHalving,
            intensifier_kwargs={
                'initial_budget': 10.0,
                'max_budget': 100,
                'eta': 2,
                'min_chall': 1},
            n_jobs=n_jobs,
            dask_client=dask_client,
        )
    return get_smac_object

automl = autosklearn.classification.AutoSklearnClassifier(
    time_left_for_this_task=120,
    per_run_time_limit=30,
    tmp_folder='/tmp/autosklearn_classification_example_tmp15',
    output_folder='/tmp/autosklearn_classification_example_out15',
    get_smac_object_callback=get_smac_object_callback("iterations")
)

def run_automl():
  automl.fit(X_train, y_train, dataset_name='breast_cancer')

  ############################################################################
  # Print the final ensemble constructed by auto-sklearn
  # ====================================================

  # print(automl.show_models())

  ###########################################################################
  # Get the Score of the final ensemble
  # ===================================

  predictions = automl.predict(X_test)
  # print("Accuracy score:", sklearn.metrics.accuracy_score(y_test, predictions))
t = threading.Thread(target=run_automl)
t.start()

history_read = 0
wait_cnt = 0
while True:
  if t.is_alive():
    print(tmp_runhistory.get_all_configs())
    # if automl.automl_ is not None and automl.automl_.cv_results_ is not None:     
    #    print(automl.automl_.cv_results_)
    #else:
    #    print("wait...")
  else:
    # clear_output(wait=True)
    # print(automl.show_models())
    predictions = automl.predict(X_test)
    print("Accuracy score:", sklearn.metrics.accuracy_score(y_test, predictions))
    break

  time.sleep(1)
